In [2]:
class tree():
            
    def __init__(self, branch_len=None, seq_leaf=None,parent=None,child=None, nuc_list=None):
        self.seq_leaf = seq_leaf
        self.child = child
        self.parent=parent
        self.branch_length = branch_len
        self.nuc_list=nuc_list
    def __len__(self):
        return len(self.seq_leaf)
        
    def getSequence(self):
        return self.seq_leaf
    
    def getBranchLength(self):
        return self.branch_length
       
    def getChildrenNodes(self):
        return self.child
    
    def get_nuc_list(self):
        return self.nuc_list
    
    def update_nuc_list(self):
        #####
        return None

In [23]:

our_Nodes=list()

current_Node=9
childA=1
childB=2
childA_BL=0.5
childB_BL=4
seq_leafA="ACGT"
seq_leafB="GGGG"

object1=tree(branch_len=childA_BL,
               seq_leaf=seq_leafA,
               parent=current_Node,
               child=childA,
               nuc_list=[])

our_Nodes.append(object1)



In [25]:
object2=tree(branch_len=childB_BL,
               seq_leaf=seq_leafB,
               parent=current_Node,
               child=childB,
               nuc_list=[])

our_Nodes.append(object2)

In [27]:
for obj in our_Nodes:
    print(obj.branch_length)
    print(obj.child)

0.5
1
4
2


In [1]:
branch_lengths_path="2023/branchlength.dat"
tree_path="2023/table.dat"
msa_path="2023/msa.dat"

with open(tree_path,"r") as tree_file, open(branch_lengths_path,"r") as BL_file, open(msa_path,"r") as msa_file:
    line_num=0
    parents=[]
    bl=BL_file.readline().split(",")
    for parent_child in tree_file:
        tmp_list=parent_child.split(",")
        parent_node_name=tmp_list[0]
        parents.append(parent_node_name)
        child_node_name=tmp_list[1]
        branch_len=bl[line_num]
        leaf_seq=None
        if not child_node_name:
            leaf_seq=
        
        
        print("parent node name: ",parent_node_name,"\n",
            "child node name: ",child_node_name,"\n",
            "bl: ",branch_len,"\n",
            "leaf_seq: ",leaf_seq,"\n")
        line_num+=1
    


SyntaxError: invalid syntax (437884213.py, line 18)

In [43]:
with  open(branch_lengths_path,"r") as BL_file:
    print(BL_file.readline().split(","))
    

['0.1', '0.4', '0.01', '0.04', '0.2', '0.08', '0.12', '0.14']
